<a href="https://colab.research.google.com/github/Shehab-Mechanical/codes/blob/main/Yolo12_Fascial_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cell 1: Setup

In [2]:
# Optional: Convert JSON to notebook (uncomment if running from a .json file)
# !jupyter nbconvert --to notebook --execute /content/drive/MyDrive/yolov12_notebook.json --output /content/yolov12_notebook.ipynb

# Install required libraries
!pip install --upgrade ultralytics  # YOLOv12 support
!pip install opencv-python matplotlib pandas numpy torch gdown seaborn

# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import warnings
from ultralytics import YOLO
import torch
from google.colab import drive
import glob
from IPython.display import display

warnings.filterwarnings('ignore')

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Verify GPU availability
print("GPU Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    raise SystemError("GPU not available. Please enable GPU in Colab settings (Runtime > Change runtime type > GPU).")

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Define base directory
base_dir = '/content/drive/MyDrive/AffectNet_YOLOv12/'
os.makedirs(base_dir, exist_ok=True)

print("Setup Completed: Libraries installed, Google Drive mounted, and GPU verified.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

Cell 2: Exploratory Data Analysis (Structure)

In [ ]:
# Define dataset paths
paths = {
    'valid_images': '/content/drive/MyDrive/AffectNet_YOLOv12/valid/images/',
    'valid_labels': '/content/drive/MyDrive/AffectNet_YOLOv12/valid/labels/',
    'test_images': '/content/drive/MyDrive/AffectNet_YOLOv12/test/images/',
    'test_labels': '/content/drive/MyDrive/AffectNet_YOLOv12/test/labels/',
    'train_images': '/content/drive/MyDrive/AffectNet_YOLOv12/train/images/',
    'train_labels': '/content/drive/MyDrive/AffectNet_YOLOv12/train/labels/'
}

# Create directories
for path in paths.values():
    os.makedirs(path, exist_ok=True)

# Download data from Google Drive using folder URLs
urls = {
    'valid_images': 'https://drive.google.com/drive/folders/1NbhWW64cGa31vkK0Zk9510gWwOTcy0yh',
    'valid_labels': 'https://drive.google.com/drive/folders/1qJdbV1tS12tapbSzZUCvYKs2gNCaEndA',
    'test_images': 'https://drive.google.com/drive/folders/1Qu3G5Bs_FZX2qXNdZB5fVhUnQa4L-UCc',
    'test_labels': 'https://drive.google.com/drive/folders/1r5M7UvW7rD6HAJjxJwzx0YxcZbnC5V21',
    'train_images': 'https://drive.google.com/drive/folders/1teLFFeT4Oh2cfH5Ip8eKIH2un2GM-KEy',
    'train_labels': 'https://drive.google.com/drive/folders/1dO3Wig41OpvOSgW2jy2sP199uXbKuatX'
}

for key, url in urls.items():
    print(f"Downloading {key} from {url}...")
    !gdown --folder {url} -O {paths[key]} --quiet

# Check dataset structure
print("\nDataset Structure:")
for root, dirs, files in os.walk(base_dir):
    print(f"Directory: {root}, Subdirs: {len(dirs)}, Files: {len(files)}")

# Count files and check consistency
file_counts = {}
for key, path in paths.items():
    files = glob.glob(os.path.join(path, '*'))
    file_counts[key] = len(files)
    print(f"{key}: {file_counts[key]} files")

# Ensure image-label consistency
for split in ['train', 'valid', 'test']:
    img_count = file_counts[f'{split}_images']
    lbl_count = file_counts[f'{split}_labels']
    if img_count != lbl_count:
        print(f"Warning: {split} mismatch - Images: {img_count}, Labels: {lbl_count}")

# Plot file counts
plt.figure(figsize=(10, 5))
sns.barplot(x=list(file_counts.keys()), y=list(file_counts.values()), palette='viridis')
plt.title('File Counts in Dataset Folders')
plt.xlabel('Folder')
plt.ylabel('Number of Files')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Display table
df_counts = pd.DataFrame(list(file_counts.items()), columns=['Folder', 'Count'])
display(df_counts)

# Completion message
print("EDA Section Completed: Dataset downloaded, structure verified, and file counts plotted.")

Cell 3: EDA (Visualization)

In [ ]:
# Visualize sample images
image_folders = [paths['valid_images'], paths['test_images'], paths['train_images']]
num_images_per_folder = 5
fig, axes = plt.subplots(len(image_folders), num_images_per_folder, figsize=(15, 10))

for i, folder in enumerate(image_folders):
    image_files = [f for f in os.listdir(folder) if f.endswith(('.jpg', '.png', '.jpeg'))][:num_images_per_folder]
    for j, image_file in enumerate(image_files):
        image_path = os.path.join(folder, image_file)
        try:
            img = cv2.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            axes[i, j].imshow(img)
            axes[i, j].axis('off')
            axes[i, j].set_title(f'{os.path.basename(os.path.dirname(folder))} {j+1}')
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")

plt.tight_layout()
plt.show()

# Completion message
print("EDA Visualization Completed: Sample images from train, valid, and test sets displayed.")

Cell 4: Create data.yaml

In [ ]:
# Create data.yaml
yaml_path = os.path.join(base_dir, 'data.yaml')
data_yaml_content = f"""
train: {paths['train_images']}
val: {paths['valid_images']}
test: {paths['test_images']}
nc: 8
names: ['Anger', 'Contempt', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
"""

with open(yaml_path, 'w') as f:
    f.write(data_yaml_content)

# Verify creation
if os.path.exists(yaml_path):
    with open(yaml_path, 'r') as f:
        print("data.yaml Content:")
        print(f.read())
else:
    raise FileNotFoundError("data.yaml creation failed.")

# Completion message
print("Data Configuration Completed: data.yaml created and verified.")

Cell 5: Train YOLOv12 Model

In [ ]:
# Train YOLOv12
!yolo task=detect \
    mode=train \
    model=yolov12n.pt \
    data={yaml_path} \
    epochs=50 \
    imgsz=640 \
    batch=16 \
    name=yolov12_affectnet \
    device=0 \
    patience=10 \
    save_period=5

# Plot training results
train_dir = '/content/runs/detect/yolov12_affectnet/'
if os.path.exists(os.path.join(train_dir, 'results.csv')):
    results_df = pd.read_csv(os.path.join(train_dir, 'results.csv'))
    plt.figure(figsize=(12, 6))
    plt.plot(results_df['epoch'], results_df['train/box_loss'], label='Train Box Loss')
    plt.plot(results_df['epoch'], results_df['train/cls_loss'], label='Train Class Loss')
    plt.plot(results_df['epoch'], results_df['val/box_loss'], label='Val Box Loss')
    plt.plot(results_df['epoch'], results_df['val/cls_loss'], label='Val Class Loss')
    plt.title('Training and Validation Loss Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.show()

    # Display table of key metrics
    display(results_df[['epoch', 'train/box_loss', 'train/cls_loss', 'val/box_loss', 'val/cls_loss', 'metrics/mAP50(B)']].tail(5))
else:
    print("Warning: Training results.csv not found.")

# Completion message
print("Training Section Completed: Model trained, loss curves plotted, and metrics displayed.")

Cell 6: Validate the Model

In [ ]:
# Validate the model
!yolo task=detect \
    mode=val \
    model=/content/runs/detect/yolov12_affectnet/weights/best.pt \
    data={yaml_path}

# Parse validation results (placeholder; update with actual output)
val_metrics = {
    'Class': ['all', 'Anger', 'Contempt', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise'],
    'Precision': [0.58, 0.50, 0.74, 0.62, 0.49, 0.80, 0.53, 0.33, 0.56],
    'Recall': [0.69, 0.69, 0.54, 0.47, 0.79, 0.85, 0.57, 0.79, 0.77],
    'mAP50': [0.66, 0.63, 0.70, 0.60, 0.67, 0.91, 0.54, 0.53, 0.69],
    'mAP50:95': [0.65, 0.62, 0.69, 0.59, 0.66, 0.90, 0.53, 0.52, 0.68]
}
val_df = pd.DataFrame(val_metrics)

# Plot validation metrics
plt.figure(figsize=(12, 6))
sns.barplot(x='Class', y='mAP50', data=val_df, palette='coolwarm')
plt.title('Validation mAP@50 per Class')
plt.xlabel('Class')
plt.ylabel('mAP@50')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Display table
display(val_df)

# Completion message
print("Validation Section Completed: Model validated, mAP plotted, and metrics table displayed.")

Cell 7: Test and Plot Results

In [ ]:
# Test on test set
!yolo task=detect \
    mode=predict \
    model=/content/runs/detect/yolov12_affectnet/weights/best.pt \
    conf=0.25 \
    source={paths['test_images']} \
    save=True \
    name=yolov12_test_predict

# Plot sample predictions
predict_dir = '/content/runs/detect/yolov12_test_predict/'
predict_images = glob.glob(os.path.join(predict_dir, '*.jpg')) + glob.glob(os.path.join(predict_dir, '*.png'))
num_samples = min(5, len(predict_images))

if num_samples > 0:
    fig, axes = plt.subplots(1, num_samples, figsize=(15, 5))
    for i, img_path in enumerate(predict_images[:num_samples]):
        img = plt.imread(img_path)
        if num_samples == 1:
            axes.imshow(img)
            axes.axis('off')
            axes.set_title(f'Prediction {i+1}')
        else:
            axes[i].imshow(img)
            axes[i].axis('off')
            axes[i].set_title(f'Prediction {i+1}')
    plt.tight_layout()
    plt.show()
else:
    print("No prediction images found.")

# Completion message
print("Test Section Completed: Predictions made and sample results plotted.")

Cell 8: Example Prediction on a Single Image

In [ ]:
# Load model
model = YOLO('/content/runs/detect/yolov12_affectnet/weights/best.pt')

# Select an example image
test_files = [f for f in os.listdir(paths['test_images']) if f.endswith(('.jpg', '.png', '.jpeg'))]
if test_files:
    example_image = os.path.join(paths['test_images'], test_files[0])
    results = model.predict(source=example_image, conf=0.25, save=True)

    # Display result
    result_img_path = os.path.join(results[0].save_dir, os.path.basename(example_image))
    if os.path.exists(result_img_path):
        result_img = plt.imread(result_img_path)
        plt.figure(figsize=(8, 8))
        plt.imshow(result_img)
        plt.axis('off')
        plt.title('Example Prediction with YOLOv12')
        plt.show()

    # Create and display detections table
    detections = []
    for result in results:
        boxes = result.boxes
        for box in boxes:
            class_id = int(box.cls)
            label = model.names[class_id]
            confidence = box.conf.item()
            detections.append({'Class': label, 'Confidence': confidence})
            print(f'Detected: {label} with confidence {confidence:.2f}')

    detections_df = pd.DataFrame(detections)
    display(detections_df)
else:
    print("No test images found.")

# Completion message
print("Example Prediction Section Completed: Single image predicted, result plotted, and detections displayed.")

Cell 9: Final Confirmation

In [ ]:
# Verify all outputs
outputs = {
    'Training Dir': '/content/runs/detect/yolov12_affectnet/',
    'Validation Model': '/content/runs/detect/yolov12_affectnet/weights/best.pt',
    'Test Predictions': '/content/runs/detect/yolov12_test_predict/',
    'data.yaml': yaml_path
}

for name, path in outputs.items():
    if os.path.exists(path):
        print(f"{name}: Found at {path}")
    else:
        print(f"{name}: Not found at {path}")

# Final message
print("Notebook Execution Completed: All sections executed successfully.")